Each team may have 1~4 students. Top 10% teams worldwide as of 11:59pm on May 11, 2022 will get A in this class.
Feature engineering: create new features from the raw data.

Supervised learning models: Choose 2 supervised learning models from W3, W4, W5.

Advanced models: choose 1 from XGBoost W12 or Neural Network W13 or Deep learning W14 to build your supervised learning models.

Model evaluation: For each of the supervised learning models, evaluate the model using the techniques from W10

If data size is too big and your model is too slow, feel free to choose small set for your project.

Your jupyter submission should be in an article quality.

Do NOT print huge data set in the notebook. Use head(). Use plotting to visualize your analysis and results. Use markdown to write your comments.

If you are not in the top 10% teams worldwide, your score is based on completeness on every step as described above.

Submit in HTML and ipynb format on canvas.<font/>

<font size='5'>Group Members:<font/>
    <br>
<font size='5'>i)Aditya Maniar(am3326) <font/>
    <br>
    <font size='5'>                       ii)Tanishq Dwivedi(td43) <font/>

In [51]:
import tensorflow as tf
import nltk
import pandas as pd
import numpy as np
import tensorflow as tf
import numpy as np 
import re
from collections import Counter
from nltk.corpus import stopwords
from nltk import ngrams
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer ,CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from tensorflow import keras
from keras.models import Sequential,Model
from keras.layers import Flatten,Dense,LSTM,Dropout,BatchNormalization,Bidirectional,Embedding, Conv1D,MaxPooling1D, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import confusion_matrix

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet') 

In [2]:
train_data=pd.read_csv('../input/nlpgettingstarted/train.csv')
test_data=pd.read_csv('../input/nlpgettingstarted/test.csv')

In [3]:
test_data

In [4]:
train_data

In [5]:
train_data.isna().sum()

In [6]:
print('Number of Unique Keywords : ',len(train_data.keyword.unique()))
train_data.keyword.value_counts(ascending=False)

In [7]:
train_data.drop('id',inplace=True,axis=1)

In [8]:
train_data.target.value_counts()

<font size="6">Preprocessing of text data</font>

In [9]:
#convert tweets to lower_case
def lower_case(text):
    text = [' '.join(tx.lower() for tx in word.split()) for word in text]
    return text

In [10]:
# Make A List Of All Words 
words=[]
for i in train_data.text:
    words.extend(i.split())
print('Unique words:',len(set(words)))
print('total words:',len(words))

In [11]:
# remove #retweet from tweets 
def retweets(text):
    tweets = [' '.join([re.sub('^@(\w)+',' ',tw) for tw in tweet.split()]) for tweet in text]  
    return tweets

In [12]:
#remove url's from tweets
def drop_url(text):
    url = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    tweets = [' '.join([re.sub(url,' ',tw) for tw in tweet.split()]) for tweet in text]  
    return tweets

In [13]:
#lemmatizing 
def lemmatize(tweets):
    lemmatizer = WordNetLemmatizer() 
    tweets = [' '.join(lemmatizer.lemmatize(word) for word in word_tokenize(tweet)) for tweet in tweets]
    return tweets


In [14]:
#removing stop words
def stop_words(tweets):
    stop_words = set(stopwords.words('english'))
    tweets = [' '.join(word for word in word_tokenize(tweet) if word not in stop_words) for tweet in tweets]
    return tweets

In [15]:
#final cleaning of tweets
def final_cleaning(tweets):
    tweets = [" ".join(word_tokenize(tweet)) for tweet in tweets]
    return tweets

In [16]:
def vectorizing(number,corpus):
    count_vec = CountVectorizer(ngram_range=(number, number)).fit(corpus)
    bag_of_words = count_vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in count_vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq

In [17]:
# get final data according to target variable ie. 0 and 1 
def final_data(target):
    clean_data = lower_case(train_data[train_data["target"] == target]["text"])
    clean_data = retweets(clean_data)
    clean_data = drop_url(clean_data)
    clean_data = lemmatize(clean_data)
    clean_data = stop_words(clean_data)
    clean_data = final_cleaning(clean_data)
    return clean_data

<font size='6'>Using Count Vectorizer to find continuous sequences( in this case '1') of words or symbols or tokens in the clean text and then printing the frequency of each word.<font/>
    <br>
    

In [18]:
#for negative reactions
vectorizing(1,final_data(0))[:50]

In [19]:
#for positive reactions
vectorizing(1,final_data(1))[:50]

<font size="6">Next we will create a pipeline with all the above functions for smooth preprocessing of the train and test data.</font> 

In [22]:
pipe1 = FunctionTransformer(lower_case)
pipe2 = FunctionTransformer(retweets)
pipe3 = FunctionTransformer(drop_url)
pipe4 = FunctionTransformer(lemmatize)
pipe5 = FunctionTransformer(final_cleaning)
pipe6 = FunctionTransformer(stop_words)

In [37]:
#This function will feed the pipeline to the different supervised models for prediction and classification report.  
def preproc_pipeline(pipe):
    pipeline = Pipeline(pipe)
    
    x =train_data.text.copy()
    y =train_data.target.copy()
    X_train ,X_test ,y_train,y_test =train_test_split(x,y,test_size=0.2,random_state=42)
    
    pipeline.fit(X_train,y_train)
    target_preds =pipeline.predict(X_test)
    print('Confusion Matrix: ')
    print(confusion_matrix(y_test, target_preds))
    print(classification_report(y_test,target_preds))
    return pipeline

<font size="6">Model 1: Using Support Vector Classification</font> 

In [38]:
SVC_model = SVC()
pipeline = [('lower',pipe1),('retweet',pipe2),('urls',pipe3),('lematize',pipe4),('spacs',pipe5),('tf_idf',TfidfVectorizer()),('model',SVC_model)]
pipeline_svc = preproc_pipeline(pipeline)

<font size='6' color='red'>Accuracy of SVC model is 82%.  <font/>
    <br>
    <br>

<font size='6'>Model 2: Using Logistic Regression  <font/>

In [39]:
logistic_reg=LogisticRegression()
pipeline = [('lower',pipe1),('retweet',pipe2),('urls',pipe3),('lematize',pipe4),('spacs',pipe5),('tf_idf',TfidfVectorizer()),('model',logistic_reg)]
pipeline_logistic_reg = preproc_pipeline(pipeline)

<font size='6' color='red'>Accuracy of Logistic Regression model is 81%. <font/>

<br>
<font size='6'>Using a CNN model<font/>

In [40]:
x = train_data.text.copy()
x = lower_case(x)
x = retweets(x)
x = drop_url(x)
x = lemmatize(x)
x = final_cleaning(x) 
y =train_data.target.copy()

tokenize = Tokenizer()
tokenize.fit_on_texts(x)
x = tokenize.texts_to_sequences(x)
print(len(tokenize.word_index)+1)
vocab_length =len(tokenize.index_word)+1
x = pad_sequences(x,maxlen=np.max(vocab_length))
X_train ,X_test ,y_train,y_test =train_test_split(x,y,test_size=0.2,random_state=42)
print(x.shape)

In [41]:
model =Sequential()

model.add(Embedding(input_length=vocab_length,output_dim=32,input_dim=vocab_length))

model.add(BatchNormalization())

model.add(GlobalMaxPooling1D())

model.add(Dropout(0.7))

model.add(Flatten())

model.add(Dense(1,activation='sigmoid'))

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=1000,
    decay_rate=0.9)
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='auto',patience=3,restore_best_weights=True)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr_schedule)
              ,loss='binary_crossentropy',metrics=['accuracy'])


In [42]:
history = model.fit(X_train,y_train,
                    epochs=15,verbose=1,validation_data = (X_test,y_test),callbacks=es)


In [43]:
model.summary()
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print('Test Accuracy: %f' % (acc*100))

In [45]:
cnn_preds=model.predict(X_test)
for i in range(0,len(cnn_preds)):
    if(cnn_preds[i]>0.5):
        cnn_preds[i]=1
    else:
        cnn_preds[i]=0
print('Confusion Matrix: ')
print(confusion_matrix(y_test, cnn_preds))
print(classification_report(y_test,cnn_preds))

<font size='6'>Prediction using CNN model<font/>

In [46]:
a = test_data.text.copy()
a = lower_case(a)
a = retweets(a)
a = drop_url(a)
a = lemmatize(a)
a = final_cleaning(a) 

tokenize = Tokenizer()
tokenize.fit_on_texts(a)
a = tokenize.texts_to_sequences(a)
print(len(tokenize.word_index)+1)
vocab_length =len(tokenize.index_word)+1
a = pad_sequences(a,maxlen=np.max(vocab_length))

In [47]:
final_preds=model.predict(a)


In [48]:
for i in range(0,len(final_preds)):
    if(final_preds[i]>0.5):
        final_preds[i]=1
    else:
        final_preds[i]=0

In [49]:
final_preds=np.array(final_preds,int)

In [ ]:
sample1 = pd.read_csv('../input/nlpgettingstarted/sample_submission.csv')
sample1['target']=final_preds
sample1.to_csv('submission1.csv',index = False)
sample1

###### <font size='6'>Prediction using SVC model<font/>

In [50]:
test_pred=test_data.text
test_prediction = pipeline_svc.predict(test_pred)

In [ ]:
sample2 = pd.read_csv('../input/nlpgettingstarted/sample_submission.csv')
sample2['target']=test_prediction
sample2.to_csv('submission2.csv',index = False)
sample2

<font size='6'>Using BertTokenizer and Bert Model for Prediction

In [ ]:
import transformers
from transformers import BertTokenizer
from transformers import TFBertModel
import random,os
import warnings

warnings.filterwarnings('ignore')


In [ ]:
SEED = 2022
def seed_everything(seed=SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
seed_everything()

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-large-uncased", do_lower_case=True)
def bert_encode(data,maximum_length) :
    input_ids = []
    attention_masks = []
    
    for i in range(len(data.text)):
        encoded = tokenizer.encode_plus(

        data.text[i],
        add_special_tokens=True,
        max_length=maximum_length,
        pad_to_max_length=True,

        return_attention_mask=True,

        )

        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])

    return np.array(input_ids),np.array(attention_masks)

In [ ]:
train_input_ids,train_attention_masks = bert_encode(train_data,60)
test_input_ids,test_attention_masks = bert_encode(test_data,60)

In [ ]:
def create_model(bert_model):
    input_ids = tf.keras.Input(shape=(60,),dtype='int32')
    attention_masks = tf.keras.Input(shape=(60,),dtype='int32')

    output = bert_model([input_ids,attention_masks])
    output = output[1]
    output = tf.keras.layers.Dense(32,activation='relu')(output)
    output = tf.keras.layers.Dropout(0.2)(output)

    output = tf.keras.layers.Dense(1,activation='sigmoid')(output)
    model = tf.keras.models.Model(inputs = [input_ids,attention_masks],outputs = output)
    model.compile(tensorflow.keras.optimizers.Adam(lr=6e-6), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
bert_model = TFBertModel.from_pretrained("bert-large-uncased")

model = create_model(bert_model)
model.summary()

In [ ]:
history = model.fit(
    [train_input_ids,train_attention_masks],
    train_data['target'],
    validation_split=0.2,
    epochs=2,
    batch_size=10)

In [ ]:
pred_test = model.predict([test_input_ids,test_attention_masks])

sub = pd.read_csv('../input/nlpgettingstarted/sample_submission.csv')
sub['target'] = np.round(pred_test).astype(int)
sub.to_csv('final_submission',index=False)
sub.head()

<font size='6'> We used SVC, Logistic Regression, CN, and a pretrained Model (Bert Model) for prediction. Since Bert Model is giving maximum accuracy while prediciting, we will use the predictions of the Bert Model for the Kaggle Competietion.<font/>